In [23]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
import fastapi
from typing import Optional


_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"]
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
    testacc=accuracy_score(y_test, y_pred)
    testrecall=recall_score(y_test, y_pred)
    testprecision=precision_score(y_test, y_pred)
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision


def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    # Assuming 'model' is defined and can predict on a DataFrame
    eval_df["predict"] = model.predict(df[[x, y]])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df


app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n:int):
    print ("RETRAINING over last {}".format(last_n))
    # Ensure _x, _y, and _result have at least last_n elements
    num_samples = min(len(_x), len(_y), len(_result))
    last_n = min(last_n, num_samples)
    # Preparing data for training
    X = np.array(list(zip(_x[-last_n:], _y[-last_n:])))  # Use zip and list to combine _x and _y
    Y = np.array(_result[-last_n:])

    # Splitting  data
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    # Trained model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model_type:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model_type, last_n, strategy, k)
    global model # Declare model as global to modify it
    if model_type == "Dummy" :
        model = DummyClassifier(strategy=strategy) # Reassign model
    elif model_type == "KNN" :
        model = KNeighborsClassifier(k=k) # Reassign model

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model_type), "train": message}
    print(returns)

    return returns

# Populating  with synthetic data (add this before calling new_model)
for i in range(50):
    _x.append(i)
    _y.append(i % 2)
    _result.append(i % 3 == 0)

result = new_model("Dummy", 50, "stratified")  # Store the result in a variable
print(result)  # Print the result

@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())

Dummy 50 stratified 0
RETRAINING over last 50
Trained Model Test Data Accuracy Score : 50.0
 
              precision    recall  f1-score   support

       False       0.50      0.70      0.58        10
        True       0.50      0.30      0.38        10

    accuracy                           0.50        20
   macro avg       0.50      0.50      0.48        20
weighted avg       0.50      0.50      0.48        20

{'message': 'Model trained Dummy', 'train': {'message': 'Model retrained', 'accuracy': 0.5, 'recall': 0.3, 'precision': 0.5}}
{'message': 'Model trained Dummy', 'train': {'message': 'Model retrained', 'accuracy': 0.5, 'recall': 0.3, 'precision': 0.5}}


#another example

In [32]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

def retrain_model(X, y, model_choice="knn", test_size=0.2, random_state=42):
    """
    Retrains a machine learning model with the given data.

    Parameters:
        X (array-like): Features (independent variables).
        y (array-like): Labels (dependent variable).
        model_choice (str): Model to train ('knn' or 'dummy').
        test_size (float): Proportion of data for testing.
        random_state (int): Seed for reproducibility.

    Returns:
        model: Trained model.
        accuracy: Accuracy score on the test set.
    """
    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # Initialize the chosen model
    if model_choice.lower() == "knn":
        model = KNeighborsClassifier(n_neighbors=3)
    elif model_choice.lower() == "dummy":
        model = DummyClassifier(strategy="stratified", random_state=random_state)
    else:
        raise ValueError("Invalid model_choice. Choose 'knn' or 'dummy'.")

    # Train the model
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return model, accuracy

#
if __name__ == "__main__":
    # Load the Breast Cancer dataset
    data = load_breast_cancer()
    X, y = data.data, data.target

    # Retraining using KNN
    knn_model, knn_accuracy = retrain_model(X, y, model_choice="knn")
    print(f"KNN Model Accuracy: {knn_accuracy:.2f}")

    # Retraining  using Dummy Classifier
    dummy_model, dummy_accuracy = retrain_model(X, y, model_choice="dummy")
    print(f"Dummy Model Accuracy: {dummy_accuracy:.2f}")


KNN Model Accuracy: 0.93
Dummy Model Accuracy: 0.60
